In [1]:
import json
from geopy.distance import great_circle
import pandas as pd
import time
from pprint import pprint
import numpy as np
from math import sin, cos, sqrt, atan2, radians


# 데이터 불러오기
# 중심좌표 데이터프레임
res7_coordinates = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/서울 전체 좌표계/res7_coordinates.csv')
res8_coordinates = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/서울 전체 좌표계/res8_coordinates.csv')
res9_coordinates = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/서울 전체 좌표계/res9_coordinates.csv')
res10_coordinates = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/서울 전체 좌표계/res10_coordinates.csv')
res11_coordinates = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/서울 전체 좌표계/res11_coordinates.csv')
res12_coordinates = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/서울 전체 좌표계/res12_coordinates.csv')
res13_coordinates = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/서울 전체 좌표계/res13_coordinates.csv')

# 학교 데이터프레임
kindergarten = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/yuchiwon.csv')
kindergarten = kindergarten.rename(columns={'유치원/latitude':'lat', '유치원/longitude':'lon'})  # 유치원 lat/lon 포함한 df

school = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/school.csv', encoding='cp949')
school = school.rename(columns={'위도':'lat', '경도':'lon'})
school_in_seoul = school.loc[school['시도교육청명'] == '서울특별시교육청']
elementary_school = school_in_seoul.loc[school_in_seoul['학교급구분'] == '초등학교']  # 초등학교 lat/lon 포함한 df
middle_school = school_in_seoul.loc[school_in_seoul['학교급구분'] == '중학교']  # 중학교 lat/lon 포함한 df
high_school = school_in_seoul.loc[school_in_seoul['학교급구분'] == '고등학교']  # 고등학교 lat/lon 포함한 df

college = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/college_coordinator.csv')
college = college.rename(columns={'Latitude':'lat', 'Longitude':'lon'})  # 대학교 lat/lon 포함한 df

# 디저트, 마라탕, 스타벅스, 탕후루, 지하철, 버스
dessert = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/dessert.csv')
dessert = dessert.rename(columns={'위도':'lat', '경도':'lon'})  # 디저트 가게 lat/lon 포함한 df

tanghulu = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/tanghulu.csv')
tanghulu = tanghulu.rename(columns={'탕후루/latitude': 'lat', '탕후루/longitude': 'lon'})  # 탕후루 가게 lat/lon 포함한 df

starbucks = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/starbucks.csv')
starbucks = starbucks.rename(columns={'스타벅스/latitude': 'lat', '스타벅스/longitude': 'lon'})  # 스타벅스 lat/lon 포함한 df

maratang = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/maratang.csv')
maratang = maratang.rename(columns={'위도':'lat', '경도':'lon'})  # 마라탕 가게 lat/lon 포함한 df

df_filtered_food = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/df_filtered_food.csv')
df_filtered_food = df_filtered_food.rename(columns={'위도':'lat', '경도':'lon'})  # 음식점 lat/lon 포함한 df

subway_station = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/subway_station.csv', encoding='cp949')
subway_station = subway_station.rename(columns={'위도':'lat', '경도':'lon'})  # 지하철역 lat/lon 포함한 df

bus_station = pd.read_excel('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/bus_station.xlsx')
bus_station = bus_station.rename(columns={'Y좌표':'lat', 'X좌표':'lon'})  # 버스정거장 lat/lon 포함한 df

# 그 외
hospital = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/hospital.csv', encoding='cp949')
hospital = hospital.rename(columns={'병원위도':'lat', '병원경도':'lon'})  # 병원 lat/lon 포함한 df

apartment = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/apart.csv', encoding='cp949')
apartment = apartment.rename(columns={'좌표Y':'lat', '좌표X':'lon'})  # 아파트 lat/lon 포함한 df
#apartment['lon'] = apartment['lon'].astype(np.float64)

bar = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/bar.csv')
bar = bar.rename(columns={'위도':'lat', '경도':'lon'})  # 술집 lat/lon 포함한 df

education = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/education_1.csv')
education = education.rename(columns={'위도':'lat', '경도':'lon'})  # 학원 lat/lon 포함한 df

lodge = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/n_lodgment.csv')
lodge = lodge.rename(columns={'latitude':'lat', 'longitude':'lon'})  # 숙박업소 lat/lon 포함한 df

movie = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/n_movie.csv')
movie = movie.rename(columns={'latitude':'lat', 'longitude':'lon'})  # 영화관 lat/lon 포함한 df

karaoke = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/n_nolaebang.csv')
karaoke = karaoke.rename(columns={'latitude':'lat', 'longitude':'lon'})  # 노래방 lat/lon 포함한 df

parking = pd.read_csv('/content/drive/MyDrive/오늘은 내가 탕후루 요리사/원본 데이터/parking.csv', encoding='cp949')
parking = parking.rename(columns={'위도':'lat', '경도':'lon'})  # 주차장 lat/lon 포함한 df

In [2]:
class CountNearThings:
  def __init__(self,df,lat,lon,dist=1):
    """
        df: 데이터 프레임
        lat: 중심 위도
        lon: 중심 경도
        dist: 기준 거리(km)"""
    self.df = df
    self.lat = lat
    self.lon = lon
    self.dist = dist

  def filter_by_rectangle(self):
      """
      사각 범위 내 데이터 필터링
      """
      lat_min = self.lat - 0.01 * self.dist
      lat_max = self.lat + 0.01 * self.dist

      lon_min = self.lon - 0.015 * self.dist
      lon_max = self.lon + 0.015 * self.dist

      # self.points = [[lat_min, lon_min], [lat_max, lon_max]]
      """
      filtered_idx = []
      lat_df = self.df['lat']
      lon_df = self.df['lon']
      for i in range(len(self.df)):
        if (self.df['lat'] > lat_min) and (self.df['lat'] < lat_max) and (self.df['lon'] > lon_min) and (self.df['lon'] < lon_max):
          filtered_idx.append[i]
      result = self.df.iloc(filtered_idx)
      """


      result = self.df[
          (self.df['lat'] > lat_min) &
          (self.df['lat'] < lat_max) &
          (self.df['lon'] > lon_min) &
          (self.df['lon'] < lon_max)
      ]

      result.index = range(len(result))

      return result

  def filter_by_radius(self, radius):
    """
    반경 범위 내 데이터 필터링
    """
  # 사각 범위 내 데이터 필터링
    tmp = self.filter_by_rectangle()  # filter_by_rectangle 거친 필터링 된 요소들로 이뤄진 DataFrame
    # pprint(tmp)

    # 기준 좌표 포인트
    center = (self.lat, self.lon)

    result_list = []
    column_names = list(tmp.columns)
    # result = pd.DataFrame()
    for index, row in tmp.iterrows():
        # 개별 좌표 포인트
        point = (row['lat'], row['lon'])
        d = great_circle(center, point).kilometers
        if d <= radius:
            # result = pd.concat([result, tmp.iloc[index, :].to_frame().T])
            result_list.append(tmp.iloc[index, :].tolist())
            # print(tmp.iloc[index, :].tolist())
            # print(type(tmp.iloc[index, :].tolist()))

    # pprint(result_list)
    result = pd.DataFrame(result_list, columns = column_names)
    result.index = range(len(result))
    # result.columns = column_names

    # pprint(result.info())

    return result

In [3]:
from math import sin, cos, sqrt, atan2, radians

def mydistance(a1,b1,a2,b2):
  R = 6373.0
  x1 = np.radians(a1)
  y1 = np.radians(b1)
  x2 = np.radians(a2)
  y2 = np.radians(b2)

  dlon = x2 - x1
  dlat = y2 - y1

  a = np.sin(dlat / 2)**2 + np.cos(y1) * np.cos(y2) * np.sin(dlon / 2)**2
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

  return R * c
  """
  R = 6373.0
  x1 = radians(a1)
  y1 = radians(b1)
  x2 = radians(a2)
  y2 = radians(b2)

  dlon = x2 - x1
  dlat = y2 - y1

  a = sin(dlat / 2)**2 + cos(y1) * cos(y2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  return R * c
  """



def weight_sum_complement(distance_list, std_dist):
  # complement = [std_dist] * len(distance_list)
  # print(np.full(len(distance_list), std_dist))
  return np.sum((np.full(len(distance_list), std_dist) - np.array(distance_list)))
"""
# 측정 거리에 따른 weight 부여 (반경 2km 기준)
# ! 0.0 km ~ 0.4 km = 3.0
# ! 0.4 km ~ 0.8 km = 2.5
# ! 0.8 km ~ 1.2 km = 2.0
# ! 1.2 km ~ 1.6 km = 1.5
# ! 1.6 km ~ 2.0 km = 1.0

def weight_sum(distance_list):

  weight_list = []

  for k in distance_list:

    if k <= 0.4:
      weight_list.append(3.0)

    elif 0.4 <= k and k <= 0.8:
     weight_list.append(2.5)

    elif 0.8 <= k and k <= 1.2:
      weight_list.append(2.0)

    elif 1.2 <= k and k <= 1.6:
     weight_list.append(1.5)

    else:
     weight_list.append(1.0)

  return sum(weight_list)
  """

'\n# 측정 거리에 따른 weight 부여 (반경 2km 기준)\n# ! 0.0 km ~ 0.4 km = 3.0\n# ! 0.4 km ~ 0.8 km = 2.5\n# ! 0.8 km ~ 1.2 km = 2.0\n# ! 1.2 km ~ 1.6 km = 1.5\n# ! 1.6 km ~ 2.0 km = 1.0\n\ndef weight_sum(distance_list):\n\n  weight_list = []\n\n  for k in distance_list:\n\n    if k <= 0.4:\n      weight_list.append(3.0)\n\n    elif 0.4 <= k and k <= 0.8:\n     weight_list.append(2.5)\n\n    elif 0.8 <= k and k <= 1.2:\n      weight_list.append(2.0)\n\n    elif 1.2 <= k and k <= 1.6:\n     weight_list.append(1.5)\n\n    else:\n     weight_list.append(1.0)\n\n  return sum(weight_list)\n  '

In [4]:
"""
# 중심 7인 위도, 경도
res7_coordinates_latitude = res7_coordinates['latitude'].values.tolist()
res7_coordinates_longitude = res7_coordinates['longitude'].values.tolist()
assert(len(res7_coordinates_latitude) == len(res7_coordinates_longitude))
"""

# 각 resolution(7~13), standard_distance(1~5) 조합 별로 계산한 feature weights값으로 이뤄진 다중 리스트 저장할 딕셔너리 선언
feature_weights_combinations = {}
'''
feature_weights_combinations :
{
  (7, 1): [
    [weight1, weigth2, weight3, ...],
    [weight1, weigth2, weight3, ...],
    :
    :
    :
  ],
  (7, 2): [
    [weight1, weigth2, weight3, ...],
    [weight1, weigth2, weight3, ...],
    :
    :
    :
  ],
  :
  :
  :
  (13, 5): [
    [weight1, weigth2, weight3, ...],
    [weight1, weigth2, weight3, ...],
    :
    :
    :
  ],
}
'''

from tqdm.notebook import tqdm
import time

# 모든 카테고리별 df 객체 담은 리스트 (iteration 돌기 위해서)
all_feature_dfs = [kindergarten, elementary_school, middle_school, high_school,
                   college, dessert, tanghulu, starbucks, maratang,
                   df_filtered_food, subway_station, bus_station, hospital,
                   apartment, bar, education, lodge, movie, karaoke, parking]

# 모든 카테고리 순서대로 이름 담은 리스트 (dataframe 만들 때 column name에 사용)
all_feature_names = ['유치원', '초등학교', '중학교', '고등학교',
                     '대학교', '디저트가게', '탕후루가게', '스타벅스', '마라탕가게',
                     '음식점', '지하철역', '버스정거장', '병원',
                     '아파트', '주점', '학원', '숙박업소', '영화관', '노래방', '주차장']


# 그리드 별로 학습용 feature weights값 계산해 DataFrame화 하기
def feature_weight_calc(all_feature_df_list, grid_coordinate_df, std_dist=2):
  # grid_lat = grid_coordinate_df['latitude'].values.tolist()  # latitude 담긴 리스트
  # grid_lon = grid_coordinate_df['longitude'].values.tolist()  # longitude 담긴 리스트
  grid_lat = grid_coordinate_df['latitude'].tolist()  # latitude 담긴 리스트
  grid_lon = grid_coordinate_df['longitude'].tolist()  # longitude 담긴 리스트
  assert len(grid_lat) == len(grid_lon)
  total_grid_length = len(grid_lat)
  print(f'Begin (Standard Distance: {std_dist})\nTotal Number of DataPoints to Compute: {total_grid_length}')
  # assert len(grid_lat) == len(grid_coordinate_df['latitude'].tolist())
  # assert len(grid_lon) == len(grid_coordinate_df['longitude'].tolist())

  feature_weights = [] # 모든 그리드 좌표들에 대해 계산된 feature weights들을 저장할 2중리스트 선언

  # for i in range(len(grid_lat)):  # 모든 그리드 기준 좌표들에 대해 반복
  for i in tqdm(range(len(grid_lat))):  # 모든 그리드 기준 좌표들에 대해 반복
    feature_weights_for_each_gridpoint = []  # i번째 그리드 지점에서 계산한 feature weights 담을 (1차원)리스트 선언
    for feature_df in all_feature_df_list:  # all_feature_df_list: 모든 feature 카테고리들의 df가 저장된 리스트
      cnt = CountNearThings(feature_df, grid_lat[i], grid_lon[i], std_dist)  # i번째 그리드 지점에서 기준거리(std_dist) 스퀘어 내의 지점만 필터해 cnt에 저장
      result_radius = cnt.filter_by_radius(std_dist)  # i번째 그리드 지점에서 기준거리(std_dist) 써클 내의 지점만 필터한 DF를 result_radius에 저장
      distance_list = []  # 기준거리로 필터링되고 남은 feature 지점과 i번째 그리드 지점 간의 실제 거리를 저장할 리스트
      # print(result_radius.info())
      # print(result_radius.shape)
      for j in range(len(result_radius)):  # 기준거리로 필터링되고 남은 feature 지점 전부에 대해 반복
        distance_list.append(mydistance(grid_lat[i],
                                        grid_lon[i],
                                        result_radius.iloc[j][result_radius.columns.get_loc('lat')],
                                        result_radius.iloc[j][result_radius.columns.get_loc('lon')]))
      feature_weights_for_each_gridpoint.append(weight_sum_complement(distance_list, std_dist))  # distance_list에 저장한 거리값을 기준으로 실제 학습에 사용할 feature weight값 계산해 feature_weights_for_each_gridpoint에 저장
    feature_weights.append(feature_weights_for_each_gridpoint)  # 모든 그리드 좌표들에 대해 계산된 feature weights들을 저장할 2차원리스트에 1차원리스트(i번째 그리드 좌표에서 계산한 featuers weights) append
    # print(feature_weights)
    # print(i)
    # print(f'{(i/total_grid_length)*100}% complete')

  return feature_weights

In [7]:
df_coordinates_by_res = {}
df_coordinates_by_res[7] = res7_coordinates
df_coordinates_by_res[8] = res8_coordinates
df_coordinates_by_res[9] = res9_coordinates
df_coordinates_by_res[10] = res10_coordinates
df_coordinates_by_res[11] = res11_coordinates
df_coordinates_by_res[12] = res12_coordinates
df_coordinates_by_res[13] = res13_coordinates

# file export directory
file_export_dir = '/content/drive/MyDrive/오늘은 내가 탕후루 요리사/training set/'

# 각 resolution(7~13), standard_distance(1~5) 조합 별로 계산한 feature weights값으로 이뤄진 다중 리스트 저장할 딕셔너리 선언
feature_weights_combinations = {}
'''
feature_weights_combinations :
{
  (7, 1): [
    [weight1, weigth2, weight3, ...],
    [weight1, weigth2, weight3, ...],
    :
    :
    :
  ],
  (7, 2): [
    [weight1, weigth2, weight3, ...],
    [weight1, weigth2, weight3, ...],
    :
    :
    :
  ],
  :
  :
  :
  (13, 5): [
    [weight1, weigth2, weight3, ...],
    [weight1, weigth2, weight3, ...],
    :
    :
    :
  ],
}
'''

# 모든 카테고리별 df 객체 담은 리스트 (iteration 돌기 위해서)
all_feature_dfs = [kindergarten, elementary_school, middle_school, high_school,
                   college, dessert, tanghulu, starbucks, maratang,
                   df_filtered_food, subway_station, bus_station, hospital,
                   apartment, bar, education, lodge, movie, karaoke, parking]

# 모든 카테고리 순서대로 이름 담은 리스트 (dataframe 만들 때 column name에 사용)
all_feature_names = ['유치원', '초등학교', '중학교', '고등학교',
                     '대학교', '디저트가게', '탕후루가게', '스타벅스', '마라탕가게',
                     '음식점', '지하철역', '버스정거장', '병원',
                     '아파트', '주점', '학원', '숙박업소', '영화관', '노래방', '주차장']

'''
df_2d = pd.DataFrame([[feature_weight_calc(all_feature_df_list=all_feature_dfs, grid_coordinate_df=df_coordinates_by_res[res], std_dist=dist) for dist in range(1, 2)] for res in range(7,8)], columns = all_feature_names)
df_2d.to_csv(file_export_dir + f'res{res}_dist{dist}_fast.csv')
'''


# file export directory
file_export_dir = '/content/drive/MyDrive/오늘은 내가 탕후루 요리사/training set/'

for res in range(8,9):  # res 범위 지정
  for dist in range(4,6):  # standard_distance: 범위 지정
    list_2d = feature_weight_calc(all_feature_df_list=all_feature_dfs,
                                   grid_coordinate_df=df_coordinates_by_res[res],
                                   std_dist=dist)
    df_2d = pd.DataFrame(list_2d)
    df_2d.columns = all_feature_names
    df_2d.to_csv(file_export_dir + f'res{res}_dist{dist}_v2.csv')

# pprint(test_2d_list)
print('done')

Begin (Standard Distance: 4)
Total Number of DataPoints to Compute: 1188


  0%|          | 0/1188 [00:00<?, ?it/s]

Begin (Standard Distance: 5)
Total Number of DataPoints to Compute: 1188


  0%|          | 0/1188 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [8]:
# 모든 카테고리별 df 객체 담은 리스트 (iteration 돌기 위해서)
all_feature_dfs_lite = [kindergarten, elementary_school, middle_school, high_school,
                   college, dessert, tanghulu, maratang,
                   subway_station, bus_station, hospital,
                   apartment, bar, education, lodge, movie, karaoke, parking]

# 모든 카테고리 순서대로 이름 담은 리스트 (dataframe 만들 때 column name에 사용)
all_feature_names_lite = ['유치원', '초등학교', '중학교', '고등학교',
                     '대학교', '디저트가게', '탕후루가게', '마라탕가게',
                     '지하철역', '버스정거장', '병원',
                     '아파트', '주점', '학원', '숙박업소', '영화관', '노래방', '주차장']

# file export directory
file_export_dir = '/content/drive/MyDrive/오늘은 내가 탕후루 요리사/training set/'


for res in range(9,10):  # res 범위 지정
  for dist in range(2,3):  # standard_distance: 범위 지정
    list_2d = feature_weight_calc(all_feature_df_list=all_feature_dfs_lite,
                                   grid_coordinate_df=df_coordinates_by_res[res],
                                   std_dist=dist)
    df_2d = pd.DataFrame(list_2d)
    df_2d.columns = all_feature_names_lite
    df_2d.to_csv(file_export_dir + f'res{res}_dist{dist}_v2_lite.csv')

# pprint(test_2d_list)
print('done')

Begin (Standard Distance: 2)
Total Number of DataPoints to Compute: 8321


  0%|          | 0/8321 [00:00<?, ?it/s]

done


In [ ]:
# 모든 카테고리별 df 객체 담은 리스트 (iteration 돌기 위해서)
all_feature_dfs_lite = [kindergarten, elementary_school, middle_school, high_school,
                   college, dessert, tanghulu, maratang,
                   subway_station, bus_station, hospital,
                   apartment, bar, education, lodge, movie, karaoke, parking]

# 모든 카테고리 순서대로 이름 담은 리스트 (dataframe 만들 때 column name에 사용)
all_feature_names_lite = ['유치원', '초등학교', '중학교', '고등학교',
                     '대학교', '디저트가게', '탕후루가게', '마라탕가게',
                     '지하철역', '버스정거장', '병원',
                     '아파트', '주점', '학원', '숙박업소', '영화관', '노래방', '주차장']

# file export directory
file_export_dir = '/content/drive/MyDrive/오늘은 내가 탕후루 요리사/training set/'


for res in range(9,10):  # res 범위 지정
  for dist in range(3,6):  # standard_distance: 범위 지정
    list_2d = feature_weight_calc(all_feature_df_list=all_feature_dfs_lite,
                                   grid_coordinate_df=df_coordinates_by_res[res],
                                   std_dist=dist)
    df_2d = pd.DataFrame(list_2d)
    df_2d.columns = all_feature_names_lite
    df_2d.to_csv(file_export_dir + f'res{res}_dist{dist}_v2_lite.csv')

# pprint(test_2d_list)
print('done')